<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E6%A0%AA%E4%BE%A1%E5%88%86%E6%9E%90%E9%96%A2%E6%95%B0_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=aa3f173c2c9c22bdbc766d64609e635b5b16c83fc3f3ceac46fcafd11ab483e4
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
import pandas_datareader.data as pdr

# 株価データを取得するメソッド
def get_stock_data(code):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :return: 株価データのデータフレーム
    """
    df = pdr.DataReader(f'{code}.JP', 'stooq').sort_index()
    return df

In [66]:
import plotly.graph_objs as go
import datetime as dt
import pandas as pd
import numpy as np
import pandas_ta as ta
# TODO: 検証用
import pprint

def show_plotly(code, name, start, end=dt.date.today(), sma=[], bbands={}, macd=[], rsi=[], stoch=[], volume=False, gcdc=False, title=''):
    """ローソク足や移動平均線のチャートを表示する

    :param code: 取得する株価データの銘柄コード
    :param code: 取得する株価データの銘柄名
    :param start: 取得する株価データの開始日
    :param end: 取得する株価データの終了日
    :param sma: チャートに移動平均線を表示するためのリスト（例: [5, 25, 75]）
    :param bbands: チャートにボリンジャーバンドを表示するための辞書（例: {'sma': 25, 'bbands': [1, 2]} or {'sma': 25, 'fill': True}）
    :param macd: チャートにMACDを表示するための短期／長期／シグナルの移動平均線の日数のリスト（例: [12, 26, 9]）
    :param rsi: チャートにRSIを表示するための移動平均線の日数のリスト（例: [14, 28]）
    :param stoch: チャートにスローストキャスティクスを表示するための移動平均線の日数のリスト（例: [14, 3, 3]）
    :param volume: チャートに出来高を表示するかどうか
    :param gcdc: チャートの移動平均線のゴールデンクロスとデッドクロスを表示するかどうか
    :param title: チャートのタイトルを表示
    """
    # 株価データの取得
    df = get_stock_data(code)
    close = df['Close']

    # ▼▼▼▼▼テクニカル指標の算出▼▼▼▼▼

    # ①移動平均線
    for ma in sma:
        df[f'ma{ma}'] = ta.sma(close, ma)
    
    # ①'ゴールデンクロスとデッドクロス
    if gcdc and len(sma) >= 2:
        cross = df[f'ma{sma[0]}'] > df[f'ma{sma[1]}']
        cross_shift = cross.shift(1)
        temp_gc = (cross != cross_shift) & (cross == True)
        temp_dc = (cross != cross_shift) & (cross == False)
        df['gc'] = [m if g == True else np.nan for g, m in zip(temp_gc, df[f'ma{sma[0]}'])]
        df['dc'] = [m if d == True else np.nan for d, m in zip(temp_dc, df[f'ma{sma[1]}'])]
    
    # ②ボリンジャーバンド
    if bbands:
        if not sma:
            df[f'ma{bbands["sma"]}'] = ta.sma(close, bbands['sma'])
        # 辞書bbandsのキーに'fill'が存在し、値がTrueかどうか確認
        if 'fill' not in bbands or not bbands['fill']:
            for n in bbands['bbands']:
                bbands_df = ta.bbands(close, length=bbands["sma"], std=n)
                df[f'lower{n}'], df[f'upper{n}'] = bbands_df.iloc[:, 0], bbands_df.iloc[:, 2]
        else:
            bbands_df = ta.bbands(close, length=bbands["sma"], std=2)
            df['lower2'], df['upper2'] = bbands_df.iloc[:, 0], bbands_df.iloc[:, 2]
    
    # ③MACD
    if macd:
        macd_df = ta.macd(close, fast=macd[0], slow=macd[1], signal=macd[2])
        df['macd'], df['hist'], df['macd_signal'] = macd_df.iloc[:, 0], macd_df.iloc[:, 1], macd_df.iloc[:, 2]
    
    # ④RSI
    if rsi:
        for ma in rsi:
            df[f'rsi{ma}'] = ta.rsi(close, ma)
        # 補助線
        df['70'], df['30'] = [70 for _ in close], [30 for _ in close]
    
    # ⑤ストキャスティクス
    if stoch:
        stoch_df = ta.stoch(df['High'], df['Low'], close, k=stoch[0], d=stoch[1], smooth_k=stoch[2])
        df['slowK'], df['slowD'] = stoch_df.iloc[:, 0], stoch_df.iloc[:, 1]
        # 補助線
        df['80'], df['20'] = [80 for _ in close], [20 for _ in close]

    # ▲▲▲▲▲テクニカル指標の算出▲▲▲▲▲
    
    # 日付で絞り込み
    cdf = df[start:end]

    # インデックスを文字列型に変更
    cdf.index = pd.to_datetime(cdf.index).strftime('%Y/%m/%d')

    # データ定義（ローソク足のデータ作成）
    data = [
        go.Candlestick(yaxis='y1', x=cdf.index, open=cdf['Open'], high=cdf['High'], low=cdf['Low'], close=cdf['Close'], name='ローソク足',
                       increasing_line_color='green',
                       increasing_line_width=1.0,
                       increasing_fillcolor='green',
                       decreasing_line_color='red',
                       decreasing_line_width=1.0,
                       decreasing_fillcolor='red')
    ]

    # チャートのタイトルを作成
    title_text = f'{code} {name}'
    title_text = f'{title_text} | {title}' if title else title_text

    # チャートに表示するために使用する変数
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    icolor = 0  # panel=0のcolorのインデックスを表す
    yaxis = 3
    yaxis_domain_spacing = .005
    # 引数にする
    yaxis_domain_raito = None
    layout_height = None

    # レイアウト定義
    layout = {
        'title': {'text': title_text, 'x': 0.5},
        'xaxis': {'rangeslider': {'visible': False}},
        'yaxis1': {'title': '価格（円）', 'side': 'left', 'tickformat': ','},
        'yaxis2': {},
        'plot_bgcolor': 'light blue'
    }

    def add_layout(title):
        """レイアウト定義にyaxis{n}の定義を追加する

        :param title: 追加するyaxisに指定するタイトル
        """
        nonlocal layout, yaxis
        yaxis_domain_raito_local = None

        def calc_domain():
            nonlocal layout, yaxis
            domain0 = .00
            domain1 = yaxis_domain_raito_local - yaxis_domain_spacing
            for n in range(yaxis, 2, -1):
                layout[f'yaxis{n}']['domain'] = [domain0, domain1]
                if n != 3:
                    domain0 = domain1 + yaxis_domain_spacing
                    domain1 = domain0 + (yaxis_domain_raito_local - yaxis_domain_spacing)
            
            domain0 = domain1 + yaxis_domain_spacing
            domain1 = domain0 + .10
            layout['yaxis2']['domain'] = [domain0, domain1]
            layout['yaxis1']['domain'] = [domain1, 1.0]

        if yaxis == 3:
            layout['height'] = layout_height if layout_height else 800
            yaxis_domain_raito_local = yaxis_domain_raito if yaxis_domain_raito else .20
            layout['yaxis3'] = {'title': title, 'side': 'right'}    # domain': [.00, .195], 
            # layout['yaxis2']['domain'] = [.20, .30]
            # layout['yaxis1']['domain'] = [.30, 1.0]
        elif yaxis == 4:
            layout['height'] = layout_height if layout_height else 875
            yaxis_domain_raito_local = yaxis_domain_raito if yaxis_domain_raito else .17
            layout['yaxis4'] = {'title': title, 'side': 'right'}    # 'domain': [.00, .165],
            # layout['yaxis3']['domain'] = [.17, .325]
            # layout['yaxis2']['domain'] = [.33, .41]
            # layout['yaxis1']['domain'] = [.41, 1.0]
        elif yaxis == 5:
            layout['height'] = layout_height if layout_height else 950
            yaxis_domain_raito_local = yaxis_domain_raito if yaxis_domain_raito else .18
            layout['yaxis5'] = {'title': title, 'side': 'right'}    # 'domain': [.00, .175],
            # layout['yaxis4']['domain'] = [.18, .355]
            # layout['yaxis3']['domain'] = [.36, .535]
            # layout['yaxis2']['domain'] = [.54, .60]
            # layout['yaxis1']['domain'] = [.60, 1.0]
        elif yaxis == 6:
            layout['height'] = layout_height if layout_height else 1250
            yaxis_domain_raito_local = yaxis_domain_raito if yaxis_domain_raito else .15
            layout['yaxis6'] = {'title': title, 'side': 'right'}    # domain': [.00, .145], 
            # layout['yaxis5']['domain'] = [.15, .295]
            # layout['yaxis4']['domain'] = [.30, .445]
            # layout['yaxis3']['domain'] = [.45, .595]
            # layout['yaxis2']['domain'] = [.60, .64]
            # layout['yaxis1']['domain'] = [.64, 1.0]
        
        calc_domain()
        yaxis += 1

    # ▼▼▼▼▼テクニカル指標のチャートへの表示▼▼▼▼▼

    # ①移動平均線
    for ma in sma:
        data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf[f'ma{ma}'], name=f'MA({ma})', line={'color': colors[icolor], 'width': 1.2}))
        icolor += 1
    
    # ①'ゴールデンクロスとデッドクロス
    if gcdc and len(sma) >= 2:
        data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf['gc'], name='ゴールデンクロス', opacity=0.5, mode='markers', marker={'size': 15, 'color': 'purple'}))
        data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf['dc'], name='デッドクロス', opacity=0.5, mode='markers', marker={'size': 15, 'color': 'black', 'symbol': 'x'}))
    
    # ②ボリンジャーバンド
    bbands_fill_between = {}
    if bbands:
        if not sma:
            data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf[f'ma{bbands["sma"]}'], name=f'MA({bbands["sma"]})', line={'color': colors[icolor], 'width': 1.2}))
            icolor += 1
        # 辞書bbandsのキーに'fill'が存在し、値がTrueかどうか確認
        if 'fill' not in bbands or not bbands['fill']:
            for n in bbands['bbands']:
                data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf[f'upper{n}'], name=f'+{n}σ', line={'color': colors[icolor], 'width': 1.2}))
                data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf[f'lower{n}'], name=f'−{n}σ', line={'color': colors[icolor], 'width': 1.2}))
                icolor += 1
        else:
            data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf['upper2'], name='', line={'color': 'lavender', 'width': 0}))
            data.append(go.Scatter(yaxis='y1', x=cdf.index, y=cdf['lower2'], name=f'±2σ', line={'color': 'lavender', 'width': 0}, fill='tonexty', fillcolor='rgba(170, 170, 170, .2)'))
            icolor += 1
    
    # ③MACD
    if macd:
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['macd'], name='MACD', line={'color': colors[0], 'width': 1}))
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['macd_signal'], name='シグナル', line={'color': colors[1], 'width': 1}))
        data.append(go.Bar(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['hist'], name='ヒストグラム', marker={'color': 'slategray'}))
        add_layout('MACD')
    
    # ④RSI
    if rsi:
        for ma, color in zip(rsi, colors):
            data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf[f'rsi{ma}'], name=f'RSI({ma})', line={'color': color, 'width': 1}))
        # 補助線
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['30'], name='30', line={'color': 'black', 'width': 1}))
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['70'], name='70', line={'color': 'black', 'width': 1}))
        add_layout('RSI')
    
    # ⑤ストキャスティクス
    if stoch:
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['slowK'], name=f'Slow%K({stoch[0]}, {stoch[1]})', line={'color': colors[0], 'width': 1}))
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['slowD'], name=f'Slow%D({stoch[2]})', line={'color': colors[1], 'width': 1}))
        # 補助線
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['20'], name='20', line={'color': 'black', 'width': 1}))
        data.append(go.Scatter(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['80'], name='80', line={'color': 'black', 'width': 1}))
        add_layout('ストキャスティクス')
    
    # ▲▲▲▲▲テクニカル指標のチャートへの表示▲▲▲▲▲

    # 出来高を表示する
    if volume:
        data.append(go.Bar(yaxis=f'y{yaxis}', x=cdf.index, y=cdf['Volume'], name='出来高', marker={'color': 'slategray'}))
        add_layout('出来高')
    
    # TODO: 検証用（削除予定）
    pprint.pprint(layout)

    # グラフ生成
    fig = go.Figure(data=data, layout=go.Layout(layout))

    # レイアウトを更新
    fig.update_layout({
        'xaxis': {
            # 日付を3日おきに表示するように設定
            'tickvals': cdf.index[::3],
        }
    })

    # 表示
    fig.show()

In [67]:
show_plotly(1928, '積水ハウス', '2021-12-1', '2022-3-31', sma=[5, 25], gcdc=True)

{'plot_bgcolor': 'light blue',
 'title': {'text': '1928 積水ハウス', 'x': 0.5},
 'xaxis': {'rangeslider': {'visible': False}},
 'yaxis1': {'side': 'left', 'tickformat': ',', 'title': '価格（円）'},
 'yaxis2': {}}


In [68]:
show_plotly(1928, '積水ハウス', '2021-12-1', '2022-3-31', sma=[5, 25], gcdc=True, bbands={'sma': 25, 'fill': True}, macd=[12, 26, 9])

{'height': 800,
 'plot_bgcolor': 'light blue',
 'title': {'text': '1928 積水ハウス', 'x': 0.5},
 'xaxis': {'rangeslider': {'visible': False}},
 'yaxis1': {'domain': [0.30000000000000004, 1.0],
            'side': 'left',
            'tickformat': ',',
            'title': '価格（円）'},
 'yaxis2': {'domain': [0.2, 0.30000000000000004]},
 'yaxis3': {'domain': [0.0, 0.195], 'side': 'right', 'title': 'MACD'}}


In [69]:
show_plotly(1928, '積水ハウス', '2021-12-1', '2022-3-31', sma=[5, 25], gcdc=True, bbands={'sma': 25, 'fill': True}, macd=[12, 26, 9], rsi=[14, 28])

{'height': 875,
 'plot_bgcolor': 'light blue',
 'title': {'text': '1928 積水ハウス', 'x': 0.5},
 'xaxis': {'rangeslider': {'visible': False}},
 'yaxis1': {'domain': [0.44000000000000006, 1.0],
            'side': 'left',
            'tickformat': ',',
            'title': '価格（円）'},
 'yaxis2': {'domain': [0.34, 0.44000000000000006]},
 'yaxis3': {'domain': [0.17, 0.335], 'side': 'right', 'title': 'MACD'},
 'yaxis4': {'domain': [0.0, 0.165], 'side': 'right', 'title': 'RSI'}}


In [70]:
show_plotly(1928, '積水ハウス', '2021-12-1', '2022-3-31', sma=[5, 25], gcdc=True, bbands={'sma': 25, 'fill': True}, macd=[12, 26, 9], rsi=[14, 28], stoch=[5, 3, 3])

{'height': 950,
 'plot_bgcolor': 'light blue',
 'title': {'text': '1928 積水ハウス', 'x': 0.5},
 'xaxis': {'rangeslider': {'visible': False}},
 'yaxis1': {'domain': [0.6399999999999999, 1.0],
            'side': 'left',
            'tickformat': ',',
            'title': '価格（円）'},
 'yaxis2': {'domain': [0.5399999999999999, 0.6399999999999999]},
 'yaxis3': {'domain': [0.36, 0.5349999999999999],
            'side': 'right',
            'title': 'MACD'},
 'yaxis4': {'domain': [0.18, 0.355], 'side': 'right', 'title': 'RSI'},
 'yaxis5': {'domain': [0.0, 0.175], 'side': 'right', 'title': 'ストキャスティクス'}}


In [71]:
show_plotly(1928, '積水ハウス', '2021-12-1', '2022-3-31', sma=[5, 25], gcdc=True, bbands={'sma': 25, 'fill': True}, macd=[12, 26, 9], rsi=[14, 28], stoch=[5, 3, 3], volume=True)

{'height': 1250,
 'plot_bgcolor': 'light blue',
 'title': {'text': '1928 積水ハウス', 'x': 0.5},
 'xaxis': {'rangeslider': {'visible': False}},
 'yaxis1': {'domain': [0.7, 1.0],
            'side': 'left',
            'tickformat': ',',
            'title': '価格（円）'},
 'yaxis2': {'domain': [0.6, 0.7]},
 'yaxis3': {'domain': [0.44999999999999996, 0.595],
            'side': 'right',
            'title': 'MACD'},
 'yaxis4': {'domain': [0.3, 0.44499999999999995],
            'side': 'right',
            'title': 'RSI'},
 'yaxis5': {'domain': [0.15, 0.295], 'side': 'right', 'title': 'ストキャスティクス'},
 'yaxis6': {'domain': [0.0, 0.145], 'side': 'right', 'title': '出来高'}}
